In [4]:
import pandas as pd
import numpy as np
import itertools

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import learning_curve, validation_curve, train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_validate, RepeatedStratifiedKFold
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import loguniform, beta, uniform

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline as IMBPipeline

import missingno as msno

import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')


In [5]:
X, y = fetch_openml(
    'titanic',
    version=1,
    as_frame=True,
    return_X_y=True
) # Load Titanic Dataset
y = y.map({'0':0,'1':1})

In [6]:
# 1. Preprocessing

class AloneTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        idx = X.sum(axis=1) > 1
        X = np.ones(X.shape[0])
        X[idx] = 0
        return X.reshape(-1,1)

    def get_feature_names_out(self, input_features=None):
        return ['is_alone']

def get_title(X):
    return X.squeeze().str.split(", ", expand=True)[1].str.split(".", expand=True)[0].values.reshape(-1,1)

class TitleExtractor(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return get_title(X)

    def get_feature_names_out(self, input_features=None):
        return ['title']


pipeline_age_fare = Pipeline([
    ('imp', KNNImputer(n_neighbors=5) ),
    ('scaler', StandardScaler() )
])

pipeline_sex_embarked = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('hot', OneHotEncoder(categories='auto',drop='first',handle_unknown='ignore'))
])

pipeline_name = Pipeline([
    ('custom', TitleExtractor()),
    ('hot', OneHotEncoder(categories='auto',drop='first',handle_unknown='ignore'))
])

final_transformation = ColumnTransformer(transformers=[
    ('age_fare', pipeline_age_fare ,['age','fare'] ),
    ('pclass', OrdinalEncoder(categories=[[1.0,2.0,3.0]]) ,['pclass'] ),
    ('sex_embarked', pipeline_sex_embarked, ['sex','embarked'] ),
    ('alone', AloneTransformer() , ['sibsp','parch']),
    ('title', pipeline_name, ['name'] )
],
remainder='drop',
verbose_feature_names_out=False,
sparse_threshold=0 #return dense matrix
)

# Note that we are not applying the transformer yet!!

In [7]:
# 2. Train and Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=30, shuffle=True)

In [8]:
# 3. Build the entire pipeline

model_pipeline = IMBPipeline([
    ('trans', final_transformation),
    ('sampler', SMOTE()),
    ('dim_reduction', PCA(n_components=0.8)),
    ('classifier', Perceptron()) # Algoritmo di Apprendimento
])

"""
Teoricamente, è parte della Pipeline anche la Trasformazione.
Posso portarla fuori se voglio ottimizzarla, dato che l'esito
è sempre quello - ma devo ricordarmi di eseguirla con
final_transformation.fit_transform(X_train)
"""

In [ ]:
# 4. Model Selection

"""
A questo punto tocca capire quale siano gli Hyper-parametri
più adeguati.
Usiamo l'approccio Cross-Validation.
> Il suo obiettivo è prevenire Overfitting (performance
pessime nel test ma buone nel training) e Underfitting (performance
pessime anche sul train set)

L'idea è di splittare ulteriormente il Training Set in Training Set e
Validation Set.
Il Validation Set serve a valutare gli iper parametri scelti
durante il training, ovvero le diverse configurazioni per il modello.

Ad esempio, potrei avere 100mila configurazioni per il modello
> Li addestro tutti sul Training Set
> Li valido tutti sul Validation Set
> Tengo solo quelli che non Underfittano e non Overfittano. Su questi faccio il Training sull'intero Train+Validation Set.
> Poi li testo tutti sul Test Set e prendo il migliore

Esistono approcci più complicati, ad esempio testando su diversi split
la configurazione e scegliendo quello che performa meglio MEDIAMENTE.

CROSS VALIDATION CON K-FOLD
- Divido il Training Set in k insiemi disgiunti, più o meno di uguale
  dimensione (si chiamano FOLD). Prima di farlo mischio i dati per
  forzare l'ipotesi IID.
- Uso k-1 fold come reale Training Set e il rimanente 1 per Validation
  Set.
- Addestro un modello per ogni split, testandone le prestazioni su
  tale split. Ottengo k valori di performance





"""



In [ ]:
"""
LEARNING E VALIDATION CURVE

Faccio il traning in batch:
- Sul primo batch misura Accuracy su training e validation set.
  Inizialmente per il training set sarà massima, perché su pochi sample è facile il fitting. Viceversa vale per il validation set.
- Aumento il numero di samples (passo un altro batch). Faccio lo stesso.

Un buon bias-variance trade-off fa sì che le curve si stringano sulla desired accuracy.


Questo è un modo per capire Overfitting o Underfitting.

Un modo per risolverli è di cambiare i parametri del modello (ridurre il numero).
"""